# Feature Extraction
* A common method of EEG classification encountered during research of literary sources was feature extraction. 
* Using the data points of the signals is not a good way to classify.
* However, extracting statistical, and signal processing related features may help significantly improve our classification results.
* There are over a hundred features that can be extracted from EEG based on the sources read.
* This notebook will try to explore a few of in the following realms:
    1. Statisical 
        * mean, median, std
    2. Time Domain
        * hjorth paramter, 1st difference mean and maximum
    3. Frequency Domain
        * Converting time domain signals into frequency domain signals with Fourier transform.
        * Caluclating band power and max power with FT.
    4. Wavelet-based features
        * Discrete Wavelet Transform (High pass and Low Pass)
    5. Auto-regressive Coefficients
        * AR model of order 6 using Burg's Algorithm. Using the coefficients as features.
* All of the feature domains will be explained further throughout the notebook.
Source: (S. S. R, J. Rabha, K. Y. Nagarjuna, D. Samanta, P. Mitra and M. Sarma, "Motor Imagery EEG Signal Processing and Classification Using Machine Learning Approach," 2017 International Conference on New Trends in Computing Sciences (ICTCS), Amman, 2017, pp. 61-66.
doi: 10.1109/ICTCS.2017.15) https://ieeexplore-ieee-org.ccny-proxy1.libr.ccny.cuny.edu/document/8250265/citations)